In [11]:
import math
import random
from collections import defaultdict
from pprint import pprint

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sns.set_style(style='white')
sns.set_context(context='notebook', font_scale=1.3, rc={'figure.figsize':(16,9)})

In [12]:
dataku = pd.read_csv('lemma.csv')

In [13]:
dataku.label.value_counts()

label
1    275
0    234
2    195
Name: count, dtype: int64

In [14]:
#proses training
import os
import pickle
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.preprocessing import LabelEncoder



In [15]:
dictio_label ={
    0:"Netral",
    1:"Positif",
    2:"Negatif"
}

dataku["label"]=dataku['label'].map(dictio_label)
dataku.label

0      Negatif
1       Netral
2      Negatif
3       Netral
4      Positif
        ...   
699    Positif
700     Netral
701    Positif
702    Positif
703     Netral
Name: label, Length: 704, dtype: object

In [16]:


dataku = dataku.dropna()
dataku.isnull().sum()

x = dataku.lemma
y = dataku.label


In [17]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(y)

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x)
train_x_tfidf = tfidf_vect.transform(x)

SVM = SVC(decision_function_shape='ovo')
text_pickle =SVM.fit(train_x_tfidf, train_y)

files = open('SVM_classifier_24.pickle','wb')
pickle.dump(text_pickle, files)
files.close()
print("selesai")

selesai


In [18]:
model = open('SVM_classifier_24.pickle', 'rb')
svm_classifier = pickle.load(model)
svm_classifier

SVC(decision_function_shape='ovo')

In [19]:
#Ekstrakasi fitur
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer

vect = TfidfVectorizer(max_features=1000, binary=True)
X = vect.fit_transform(dataku.lemma)

print(X)


#sm=SMOTE()

  (0, 298)	0.31398902087385483
  (0, 536)	0.3702590525826746
  (0, 469)	0.3702590525826746
  (0, 506)	0.34095306078162635
  (0, 653)	0.3304932995637637
  (0, 395)	0.3072318252347618
  (0, 933)	0.3304932995637637
  (0, 11)	0.321649697463621
  (0, 297)	0.062952476436725
  (0, 873)	0.30118730776271546
  (1, 688)	0.2937241608778118
  (1, 784)	0.3267418423316161
  (1, 320)	0.25509594985534795
  (1, 567)	0.2937241608778118
  (1, 867)	0.28309484158979076
  (1, 503)	0.28309484158979076
  (1, 655)	0.3267418423316161
  (1, 327)	0.28309484158979076
  (1, 119)	0.25509594985534795
  (1, 791)	0.3267418423316161
  (1, 11)	0.2670671732411528
  (1, 297)	0.05226972095127789
  (1, 873)	0.25007716013598646
  (2, 992)	0.44368376175643987
  (2, 998)	0.44368376175643987
  :	:
  (696, 297)	0.051527571586569704
  (697, 92)	0.6184108466381596
  (697, 91)	0.5002032900690235
  (697, 637)	0.5960317331173387
  (697, 297)	0.11004938201344236
  (698, 865)	0.6716931671007793
  (698, 311)	0.5696127573235762
  (698, 879

In [20]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

X = dataku.lemma
Y = dataku.label

ss = ShuffleSplit(n_splits=10, test_size=0.2)
sm = SMOTE()

accs = []
fis = []
cms = []

for train_index, test_index in ss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)

    X_train_res, y_train_res = sm.fit_resample(X_train_vect, y_train)
    
    SVM.fit(X_train_res, y_train_res)
    y_pred = SVM.predict(X_test_vect)

    accs.append(accuracy_score(y_test,y_pred))
    fis.append(f1_score(y_test, y_pred, average='weighted'))
    cms.append(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))
print("\naverage accuracy across fold : {:2f}%".format(sum(accs)/len(accs) *100))
print("\naverage F1-score across fold : {:2f}%".format(sum(fis)/len(fis) *100))
print("\naverage Confusion Matrix across fold : \n {}".format(sum(cms)/len(cms)))

              precision    recall  f1-score   support

     Negatif       0.56      0.48      0.51        42
      Netral       0.56      0.59      0.57        46
     Positif       0.70      0.75      0.72        52

    accuracy                           0.61       140
   macro avg       0.60      0.60      0.60       140
weighted avg       0.61      0.61      0.61       140


average accuracy across fold : 62.928571%

average F1-score across fold : 62.665007%

average Confusion Matrix across fold : 
 [[20.7 13.2  8.1]
 [ 8.3 28.9  7.6]
 [ 5.6  9.1 38.5]]
